In [ ]:
!pip install datasets
!pip install langchain
!pip install ctransformers
!pip install nltk
!pip install faiss-cpu
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.2 MB/s eta 0:00:00


In [ ]:
import re, nltk
from datasets import load_dataset
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
dataset = load_dataset("hugginglearners/netflix-shows", split="train")
df = pd.DataFrame(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(df)

     show_id     type                  title         director  \
0         s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1         s2  TV Show          Blood & Water             None   
2         s3  TV Show              Ganglands  Julien Leclercq   
3         s4  TV Show  Jailbirds New Orleans             None   
4         s5  TV Show           Kota Factory             None   
...      ...      ...                    ...              ...   
8802   s8803    Movie                 Zodiac    David Fincher   
8803   s8804  TV Show            Zombie Dumb             None   
8804   s8805    Movie             Zombieland  Ruben Fleischer   
8805   s8806    Movie                   Zoom     Peter Hewitt   
8806   s8807    Movie                 Zubaan      Mozez Singh   

                                                   cast        country  \
0                                                  None  United States   
1     Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa 

In [ ]:
print(df.columns)

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Data preprocessing
X = df['description']
y = df['listed_in']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the textual data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test) # Text data --> numerical feature vectors to feed into ML

# Train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

# Evaluate the classifier
y_pred = clf.predict(X_test_tfidf)
# print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=1))

                                                                                 precision    recall  f1-score   support

                                                             Action & Adventure       1.00      0.00      0.00        27
                   Action & Adventure, Anime Features, Children & Family Movies       1.00      0.00      0.00         1
                             Action & Adventure, Anime Features, Classic Movies       1.00      0.00      0.00         1
                       Action & Adventure, Anime Features, International Movies       1.00      0.00      0.00         8
                           Action & Adventure, Anime Features, Sci-Fi & Fantasy       1.00      0.00      0.00         1
                   Action & Adventure, Children & Family Movies, Classic Movies       1.00      0.00      0.00         1
                         Action & Adventure, Children & Family Movies, Comedies       1.00      0.00      0.00         1
                 Action & Adven

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Data preprocessing
X = df['description']
y = df['listed_in']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the textual data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a logistic regression classifier
clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)

# Evaluate the classifier
y_pred = clf.predict(X_test_tfidf)
# print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=1))


                                                                                 precision    recall  f1-score   support

                                                             Action & Adventure       0.38      0.22      0.28        27
                   Action & Adventure, Anime Features, Children & Family Movies       1.00      0.00      0.00         1
                             Action & Adventure, Anime Features, Classic Movies       1.00      0.00      0.00         1
                       Action & Adventure, Anime Features, International Movies       1.00      0.00      0.00         8
                           Action & Adventure, Anime Features, Sci-Fi & Fantasy       1.00      0.00      0.00         1
                   Action & Adventure, Children & Family Movies, Classic Movies       1.00      0.00      0.00         1
                         Action & Adventure, Children & Family Movies, Comedies       1.00      0.00      0.00         1
                 Action & Adven

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Define the custom evaluation function
def evaluate_classifier(y_true, y_pred):
    # Convert multi-label genre strings into lists
    y_true_list = [true.split(', ') for true in y_true]
    y_pred_list = [pred.split(', ') for pred in y_pred]

    # Initialize empty lists to store binary classification results
    correct_predictions = []

    # Iterate over each instance
    for true, pred in zip(y_true_list, y_pred_list):
        # Check if any predicted genre matches with any true genre
        match = any([p in true for p in pred])
        correct_predictions.append(match)

    # Calculate precision, recall, and F1-score
    precision = sum(correct_predictions) / len(correct_predictions)
    recall = sum(correct_predictions) / len(y_true)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

# Load the dataset
dataset = load_dataset("hugginglearners/netflix-shows", split="train")
df = pd.DataFrame(dataset)

# Data preprocessing
X = df['description']
y = df['listed_in']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the textual data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a logistic regression classifier
clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = clf.predict(X_test_tfidf)

# Evaluate the classifier using custom evaluation function
precision, recall, f1 = evaluate_classifier(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Precision: 0.49375709421112374
Recall: 0.49375709421112374
F1-score: 0.49375709421112374


In [ ]:
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from datasets import load_dataset
import random

# Function to generate embeddings
def generate_embeddings(descriptions: list) -> Tensor:
    tokenizer = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-Mistral')
    model = AutoModel.from_pretrained('Salesforce/SFR-Embedding-Mistral')
    max_length = 4096
    batch_dict = tokenizer(descriptions, max_length=max_length, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**batch_dict)
    embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)
    return embeddings

# Function to find matches between LLM-generated descriptions and real data descriptions
def find_matches(llm_generated_descriptions: list, real_data_descriptions: list, threshold: float) -> list:
    llm_generated_embeddings = generate_embeddings(llm_generated_descriptions)
    real_data_embeddings = generate_embeddings(real_data_descriptions)

    similarity_matrix = cosine_similarity(llm_generated_embeddings, real_data_embeddings)

    matches = []
    for i, llm_embedding in enumerate(llm_generated_embeddings):
        for j, real_embedding in enumerate(real_data_embeddings):
            similarity_score = similarity_matrix[i, j]
            if similarity_score > threshold:
                matches.append((llm_generated_descriptions[i], real_data_descriptions[j], similarity_score))
    return matches

# Simulated LLM-generated descriptions for horror genre
def generate_llm_descriptions(genre: str, num_descriptions: int) -> list:
    llm_generated_descriptions = []
    for _ in range(num_descriptions):
        llm_generated_descriptions.append(f"Generate {genre} description here...")
    return llm_generated_descriptions

# Load the Netflix dataset
netflix_dataset = load_dataset("hugginglearners/netflix-shows", split="train")
real_data_descriptions = netflix_dataset['description']

# Initialize the bucket queue of potential genres
genre_queue = ["horror", "comedy", "mystery"]  # Example genres

# Iterate over each genre in the queue
for genre in genre_queue:
    # Simulate LLM generating descriptions for the current genre
    llm_generated_descriptions = generate_llm_descriptions(genre, num_descriptions=25)  # Generate 25 descriptions for each genre

    # Find matches between LLM-generated descriptions and real data descriptions for the current genre
    matches = find_matches(llm_generated_descriptions, real_data_descriptions, threshold=0.8)  # Adjust threshold as needed

    # Print matches
    print(f"Matches for {genre} genre:")
    for llm_description, real_description, similarity_score in matches:
        print(f"LLM Description: {llm_description}")
        print(f"Real Data Description: {real_description}")
        print(f"Similarity Score: {similarity_score}")
        print()

# Utility function to extract last token embeddings
def last_token_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]


tokenizer_config.json:   0%|          | 0.00/981 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.28G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]